In [ ]:
lab10-2

# lab 10-2


In [20]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
import datetime

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [72]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """ #초모수값 지정
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size #468
label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size).\
    repeat()

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x)).\
    repeat()

#train_iterator = train_dataset.make_one_shot_iterator()
#test_iterator = test_dataset.make_one_shot_iterator()

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [82]:
###수정!!
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        ###Xavier initialization 이용
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Flatten(input_shape=(28,28,)))
        self.model.add(tf.keras.layers.Dense(256,kernel_initializer=weight_init, input_shape=(784,),activation="relu"))
        self.model.add(tf.keras.layers.Dense(256,kernel_initializer= weight_init)) ##model형성시 input shape 지정 해줄 것, 이 경우 28*28input을  flatten 시키므로 
        self.model.add(relu())

        self.model.add(tf.keras.layers.Dense(label_dim, kernel_initializer=weight_init))
        model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x
    


In [104]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256,kernel_initializer=weight_init, input_shape=(784,),activation="relu"))
    model.add(tf.keras.layers.Dense(256, kernel_initializer=weight_init)) ##model형성시 input shape 지정 해줄 것, 이 경우 28*28input을  flatten 시키므로 
    model.add(relu())
    model.add(tf.keras.layers.Dense(label_dim,kernel_initializer= weight_init))
    
    model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    return model

model = create_model_function(label_dim)
#model.summary()

In [107]:
#이걸로 실행시 x --> 왜?
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(28,28,1)))
    model.add(tf.keras.layers.Dense(256,kernel_initializer=weight_init, input_shape=(784,),activation="relu"))
    model.add(tf.keras.layers.Dense(256, kernel_initializer=weight_init)) ##model형성시 input shape 지정 해줄 것, 이 경우 28*28input을  flatten 시키므로 
    model.add(relu())
    model.add(tf.keras.layers.Dense(label_dim,kernel_initializer= weight_init))
    
    model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    return model

model = create_model_function(label_dim)
model.summary()

Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_37 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_81 (Dense)             (None, 256)               200960    
_________________________________________________________________
dense_82 (Dense)             (None, 256)               65792     
_________________________________________________________________
activation_33 (Activation)   (None, 256)               0         
_________________________________________________________________
dense_83 (Dense)             (None, 10)                2570      
Total params: 269,322
Trainable params: 269,322
Non-trainable params: 0
_________________________________________________________________


In [105]:

""" Model """
network = create_model_function(label_dim)

""" Training """
#optimizer =tf.keras.optimizers.Adam(learning_rate)

""" Writer """

model_dir = 'nn_xavier'


In [108]:
#for checkpoint
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_10_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 체크포인트 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

network.fit(train_dataset, epochs =1,
          validation_data =test_dataset,steps_per_epoch=training_iterations,
          callbacks = [cp_callback],validation_steps=1)  # 훈련 단계에 콜백을 전달합니다

# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.

466/468 [============================>.] - ETA: 0s - loss: 8.2460 - accuracy: 0.2085
Epoch 00001: saving model to training_10_2/cp-0001.ckpt
468/468 [==============================] - 6s 13ms/step - loss: 8.2436 - accuracy: 0.2086 - val_loss: 8.2315 - val_accuracy: 0.2081


============================================================================================================

In [ ]:
##for tensorboard

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = 'logs/gradient_tape/' + current_time 
summary_writer = tf.summary.create_file_writer(log_dir)
callbacks = [
  # `val_loss`가 2번의 에포크에 걸쳐 향상되지 않으면 훈련을 멈춥니다.
  tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),

    # `./logs` 디렉토리에  로그를 기록니다.
  tf.keras.callbacks.TensorBoard(log_dir=log_dir)
]


if train_flag :
    tf.saved_model.load(path)
    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        start_iteration = checkpoint_counter - start_epoch * training_iterations
        counter = checkpoint_counter
        global_step.assign(checkpoint_counter)
        print(" [*] Load SUCCESS")

    
    # train phase
    cord_summaries():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx in range(start_iteration, training_iterations):
                train_input, train_label = train_iterator.get_next()
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables), global_step=global_step)

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                test_input, test_label = test_iterator.get_next()
                test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.contrib.summary.scalar(name='train_loss', tensor=train_loss)
                tf.contrib.summary.scalar(name='train_accuracy', tensor=train_accuracy)
                tf.contrib.summary.scalar(name='test_accuracy', tensor=test_accuracy)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        tf.saved_model.save(exported, path)
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    test_input, test_label = test_iterator.get_next()
    test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))